In [6]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [7]:
url_unique ='https://www.superga.com'
url_base = 'https://www.superga.com/collections/'
url_page = 'womens-slippers'
url = url_base+url_page
header = {"User-Agent":"Chrome/39.0"}
print(url)
page = requests.get(url, headers=header)
print(page)

https://www.superga.com/collections/womens-slippers
<Response [200]>


In [8]:
soup = BeautifulSoup(page.text)
ciabattenomi = soup.find_all('a', class_="grid-product__link")

print(ciabattenomi)

[<a class="grid-product__link" href="/collections/womens-slippers/products/slippers-unisex-1908-slides-clear-identity-black-s51138w-005">
<div class="grid-product__meta cb-text-left">
<div class="grid-product__title grid-product__title--heading">   1908 SLIDES CLEAR IDENTITY<span class="cb-italic cb-block cb-font-500">BLACK</span></div><div class="grid-product__price">
<span class="">
  
      from
    
  
    €35,00
  
</span>
</div></div>
</a>, <a class="grid-product__link" href="/collections/womens-slippers/products/slippers-unisex-1908-slides-clear-identity-pink-tickled-s51138w-xt8">
<div class="grid-product__meta cb-text-left">
<div class="grid-product__title grid-product__title--heading">   1908 SLIDES CLEAR IDENTITY<span class="cb-italic cb-block cb-font-500">PINK TICKLED</span></div><div class="grid-product__price">
<span class="">
  
      from
    
  
    €35,00
  
</span>
</div></div>
</a>, <a class="grid-product__link" href="/collections/womens-slippers/products/slippers-un

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [10]:
rows = []
for product in ciabattenomi:
    try:
        # Get the product URL
        url_ciabatta = product.get('href')  # Get the link from ciabattenomi
        url1 = url_unique + url_ciabatta  # Construct the full URL
        driver.get(url1)  # Load the product page

        # Wait for JavaScript to render the page
        time.sleep(5)

        # Extract the rendered HTML
        html_content = driver.page_source
        soup_ciabatta = BeautifulSoup(html_content, 'html.parser')

        # Extract product details
        name_ciabatta = soup_ciabatta.find('h1', class_='h2 product-single__title')
        colore_ciabatta = soup_ciabatta.find('span', class_='cb-italic cb-block cb-font-500')
        prezzo_originale_ciabatta = soup_ciabatta.find('span', class_='related__price mr')
        colors_divs = soup_ciabatta.find_all('div', class_='related__color-name skeleton-effect')
        sizes = soup_ciabatta.find('ul', class_='related__select-optgroup-options')

        nomi = name_ciabatta.text.strip()
        colore =colore_ciabatta.text.strip()
        prezzo_originale = prezzo_originale_ciabatta.text.strip()
        
        # Remove color info from the name
        if colore in nomi:
            nomi = nomi.replace(colore, "")

        all_colors = []
        
        # Get all colors
        for color_div in colors_divs:
            color_name = color_div.text.strip()
            if color_name:  # Ensure the color name is not empty
                all_colors.append(color_name)

        # Get all sizes
        all_size = []

        for size in sizes :
            size_val = size.text.strip()
            if size :
                all_size.append(size_val)
        print(nomi)
        print(colore)
        print(prezzo_originale)
        print(all_colors)
        print(all_size)
        # Loop through all colors and sizes to add combinations to rows
        for color in all_colors:
            for size in all_size:
                rows.append({
                    'name': nomi,                 # Product name
                    'color': color,               # Color option
                    'size': size[:2],                 # Size option
                    'price_sale': "",                   # Sale price
                    'price': prezzo_originale[1:]      # Original price
                })
        print(rows)
    except Exception as e:
        print(f"Error processing product: {product}. Error: {e}")

# Create DataFrame from the rows
df = pd.DataFrame(rows)


# Display the DataFrame
print(df)

1908 SLIDES CLEAR IDENTITY
BLACK
€35.00
['BLACK', 'PINK TICKLED', 'WHITE AVORIO-VIOLET LILLA']
['35Only 5 available', '36', '37', '38', '39', '40', '41Only 2 available', '42Out of stock', '43Out of stock', '44Out of stock', '45Out of stock', '46Out of stock']
[{'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '35', 'price_sale': '', 'price': '35.00'}, {'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '36', 'price_sale': '', 'price': '35.00'}, {'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '37', 'price_sale': '', 'price': '35.00'}, {'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '38', 'price_sale': '', 'price': '35.00'}, {'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '39', 'price_sale': '', 'price': '35.00'}, {'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '40', 'price_sale': '', 'price': '35.00'}, {'name': '1908 SLIDES CLEAR IDENTITY', 'color': 'BLACK', 'size': '41', 'price_sale': '', 'p

In [11]:
# Save the DataFrame to a CSV file
df.to_csv('db_ciabatte.csv', index=False)